In [ ]:
# Модифицировать реализацию KNN  так, 
#чтобы больший вес получали наиболее близко расположенные объекты(использовать вес 1/d или 1/max(const, d)).
import numpy as np
from sklearn import model_selection
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from matplotlib.colors import ListedColormap

X, y = load_iris(return_X_y=True)

# Для наглядности возьмем только первые два признака (всего в датасете их 4)
X = X[:, :2]

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=1)
cmap = ListedColormap(['red', 'green', 'blue'])
plt.figure(figsize=(7, 7))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap)

In [ ]:
def e_metrics(x1, x2):
    
    distance = 0
    for i in range(len(x1)):
        distance += np.square(x1[i] - x2[i])
    
    return np.sqrt(distance)

In [ ]:
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

In [ ]:
def knn_weighted(x_train, y_train, x_test, k):
    
    answers = []
    for x in x_test:
        test_distances = []
            
        for i in range(len(x_train)):
            
            # расчет расстояния от классифицируемого объекта до
            # объекта обучающей выборки
            distance = e_metrics(x, x_train[i])
            
            # Записываем в список значение расстояния и ответа на объекте обучающей выборки
            test_distances.append((distance, y_train[i]))
        
        # создаем словарь со всеми возможными классами
        classes = {class_item: 0 for class_item in set(y_train)}
        
        # Сортируем список и среди первых k элементов подсчитаем частоту появления разных классов
        for index, d in enumerate(sorted(test_distances)[0:k], start=1):
            classes[d[1]] += 1/index
            
        # Записываем в список ответов наиболее часто встречающийся класс
        answers.append(sorted(classes, key=classes.get)[-1])
    return answers

In [ ]:
def knn(x_train, y_train, x_test, k):
    
    answers = []
    for x in x_test:
        test_distances = []
            
        for i in range(len(x_train)):
            
            # расчет расстояния от классифицируемого объекта до
            # объекта обучающей выборки
            distance = e_metrics(x, x_train[i])
            
            # Записываем в список значение расстояния и ответа на объекте обучающей выборки
            test_distances.append((distance, y_train[i]))
        
        # создаем словарь со всеми возможными классами
        classes = {class_item: 0 for class_item in set(y_train)}
        
        # Сортируем список и среди первых k элементов подсчитаем частоту появления разных классов
        for d in sorted(test_distances)[0:k]:
            classes[d[1]] += 1
            
        # Записываем в список ответов наиболее часто встречающийся класс
        answers.append(sorted(classes, key=classes.get)[-1])
    return answers

In [ ]:
def get_graph(X_train, y_train, k, weighted=False):
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA','#00AAFF'])

    h = .02

    # Расчет пределов графика
    x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
    y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # Получим предсказания для всех точек
    if weighted:
        Z = knn_weighted(X_train, y_train, np.c_[xx.ravel(), yy.ravel()], k)
    else:
        Z = knn(X_train, y_train, np.c_[xx.ravel(), yy.ravel()], k)

    # Построим график
    Z = np.array(Z).reshape(xx.shape)
    plt.figure(figsize=(7,7))
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Добавим на график обучающую выборку
    plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cmap)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title(f"Трехклассовая kNN классификация при k = {k}")
    plt.show()

In [ ]:
def predictions_tuning(X_train, y_train, X_test, k_list):
    for k in k_list:
        y_pred = knn(X_train, y_train, X_test, k)
        print(f'Точность алгоритма kNN при k = {k}: {accuracy(y_pred, y_test):.3f}')
        y_pred = knn_weighted(X_train, y_train, X_test, k)
        print(f'Точность алгоритма kNN weighted при k = {k}: {accuracy(y_pred, y_test):.3f}\n')

In [ ]:
k_list = [i for i in range(1, 16) if i%2!=0]
predictions_tuning(X_train, y_train, X_test, k_list)

In [ ]:
Точность алгоритма kNN при k = 1: 0.667
Точность алгоритма kNN weighted при k = 1: 0.667

Точность алгоритма kNN при k = 3: 0.733
Точность алгоритма kNN weighted при k = 3: 0.667

Точность алгоритма kNN при k = 5: 0.867
Точность алгоритма kNN weighted при k = 5: 0.733

Точность алгоритма kNN при k = 7: 0.800
Точность алгоритма kNN weighted при k = 7: 0.733

Точность алгоритма kNN при k = 9: 0.867
Точность алгоритма kNN weighted при k = 9: 0.767

Точность алгоритма kNN при k = 11: 0.833
Точность алгоритма kNN weighted при k = 11: 0.767

Точность алгоритма kNN при k = 13: 0.833
Точность алгоритма kNN weighted при k = 13: 0.767

Точность алгоритма kNN при k = 15: 0.833
Точность алгоритма kNN weighted при k = 15: 0.767

Вывод: в этом кейсе взвешенный kNN показал accuracy ниже, чем базовый алгоритм kNN, который просто считает число соседей.